In [ ]:
# import os
# import cv2
# import matplotlib.pyplot as plt

# # Paths to your folders
# dataset_path = "/Volumes/Exodus/Data/ART-Net/"

# train_positive_images_path = os.path.join(dataset_path, "Train/Train_Positive")
# train_negative_images_path = os.path.join(dataset_path, "Train/Train_Negative")
# train_masks_path = os.path.join(dataset_path, "Train/Train_Positive_Tool_Mask")
# train_labels_path = os.path.join(dataset_path, "labels/train")

# test_positive_images_path = os.path.join(dataset_path, "Test/Test_Positive")
# test_negative_images_path = os.path.join(dataset_path, "Test/Test_Negative")
# test_masks_path = os.path.join(dataset_path, "Test/Test_Positive_Tool_Mask")
# test_labels_path = os.path.join(dataset_path, "labels/val")

# os.makedirs(train_labels_path, exist_ok=True)
# os.makedirs(test_labels_path, exist_ok=True)

# # Function to create YOLO annotations from masks
# def create_annotations(images_path, masks_path, labels_path, n = None):
#     if n is not None:
#         image_files = sorted([f for f in os.listdir(images_path) if f.endswith(".png") and not f.startswith("._")])[:n]
#     else:
#         image_files = sorted([f for f in os.listdir(images_path) if f.endswith(".png") and not f.startswith("._")])

#     for image_file in image_files:
#         image_path = os.path.join(images_path, image_file)
#         mask_file = image_file.replace(".png", "_Tool_Mask.png")
#         mask_path = os.path.join(masks_path, mask_file)
        
#         # print(image_path)
#         # print(mask_path)

#         if os.path.exists(mask_path):
#             # print(f"Processing {image_file}...")
#             # Load the mask image
#             mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
#             contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#             # Open the corresponding label file
#             label_file = os.path.join(labels_path, image_file.replace(".png", ".txt"))
#             with open(label_file, "w") as lf:
#                 for contour in contours:
#                     x, y, w, h = cv2.boundingRect(contour)
#                     # Normalize coordinates
#                     x_center = (x + w / 2) / mask.shape[1]
#                     y_center = (y + h / 2) / mask.shape[0]
#                     width = w / mask.shape[1]
#                     height = h / mask.shape[0]
#                     # Write to label file in YOLO format
#                     lf.write(f"0 {x_center} {y_center} {width} {height}\n")

#                 # # use matplotlib to display the images with bounding box
#                 # image = cv2.imread(image_path)
#                 # for contour in contours:
#                 #     x, y, w, h = cv2.boundingRect(contour)
#                 #     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
#                 # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#                 # plt.axis("off")
#                 # plt.show()

#     print("Annotations created in YOLO format.")

# # Create annotations for training images
# create_annotations(train_positive_images_path, train_masks_path, train_labels_path)
# # Create annotations for test images
# create_annotations(test_positive_images_path, test_masks_path, test_labels_path)

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Paths to your folders
dataset_path = "/Volumes/Exodus/Data/ART-Net/"

train_positive_images_path = os.path.join(dataset_path, "Train/Train_Positive")
train_negative_images_path = os.path.join(dataset_path, "Train/Train_Negative")
train_masks_path = os.path.join(dataset_path, "Train/Train_Positive_Tool_Mask")
train_tooltip_masks_path = os.path.join(dataset_path, "Train/Train_Positive_TipPoint")
train_labels_path = os.path.join(dataset_path, "labels/train")

test_positive_images_path = os.path.join(dataset_path, "Test/Test_Positive")
test_negative_images_path = os.path.join(dataset_path, "Test/Test_Negative")
test_masks_path = os.path.join(dataset_path, "Test/Test_Positive_Tool_Mask")
test_tooltip_masks_path = os.path.join(dataset_path, "Test/Test_Positive_TipPoint")
test_labels_path = os.path.join(dataset_path, "labels/val")

os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(test_labels_path, exist_ok=True)


# Function to create YOLO annotations from masks
def create_annotations(
    images_path, masks_path, tooltip_masks_path, labels_path, n=None
):
    if n is not None:
        image_files = sorted(
            [
                f
                for f in os.listdir(images_path)
                if f.endswith(".png") and not f.startswith("._")
            ]
        )[:n]
    else:
        image_files = sorted(
            [
                f
                for f in os.listdir(images_path)
                if f.endswith(".png") and not f.startswith("._")
            ]
        )

    for image_file in image_files:
        image_path = os.path.join(images_path, image_file)
        mask_file = image_file.replace(".png", "_Tool_Mask.png")
        mask_path = os.path.join(masks_path, mask_file)
        tooltip_mask_file = image_file.replace(".png", "_TipPoint.png")
        tooltip_mask_path = os.path.join(tooltip_masks_path, tooltip_mask_file)

        label_file = os.path.join(labels_path, image_file.replace(".png", ".txt"))

        with open(label_file, "w") as lf:
            if os.path.exists(mask_path):
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                contours, _ = cv2.findContours(
                    mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
                )
                for contour in contours:
                    contour = contour.squeeze()  # Remove extra dimension
                    if len(contour.shape) == 1:  # Single point, duplicate to make a box
                        contour = np.array([contour, contour])
                    contour = contour.reshape(-1)
                    normalized_contour = []
                    for i in range(0, len(contour), 2):
                        x = contour[i] / mask.shape[1]
                        y = contour[i + 1] / mask.shape[0]
                        normalized_contour.extend([x, y])
                    lf.write(f"0 {' '.join(map(str, normalized_contour))}\n")

            # if os.path.exists(tooltip_mask_path):
            #     tooltip_mask = cv2.imread(tooltip_mask_path, cv2.IMREAD_GRAYSCALE)
            #     tooltip_contours, _ = cv2.findContours(
            #         tooltip_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            #     )
            #     for contour in tooltip_contours:
            #         contour = contour.squeeze()  # Remove extra dimension
            #         if len(contour.shape) == 1:  # Single point, duplicate to make a box
            #             contour = np.array([contour, contour])
            #         contour = contour.reshape(-1)
            #         normalized_contour = []
            #         for i in range(0, len(contour), 2):
            #             x = contour[i] / tooltip_mask.shape[1]
            #             y = contour[i + 1] / tooltip_mask.shape[0]
            #             normalized_contour.extend([x, y])
            #         lf.write(f"1 {' '.join(map(str, normalized_contour))}\n")
    
            # Use matplotlib to draw the images with bounding box
            image = cv2.imread(image_path)
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
            # for contour in tooltip_contours:
            #     x, y, w, h = cv2.boundingRect(contour)
            #     cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
                
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.axis("off")

    print("Annotations created in YOLO format.")


# Create annotations for training images
# create_annotations(
#     train_positive_images_path,
#     train_masks_path,
#     train_tooltip_masks_path,
#     train_labels_path,
# )

# # Create annotations for test images
# create_annotations(
#     test_positive_images_path,
#     test_masks_path,
#     test_tooltip_masks_path,
#     test_labels_path,
# )

In [4]:
from ultralytics import YOLO

# Create a configuration file for YOLOv8
config_content = f"""
train: {dataset_path}/images/train
val: {dataset_path}/images/val

nc: 1  # number of classes
names: ['tool']  # class names
"""

# config_content = f"""
# train: {dataset_path}/images/train
# val: {dataset_path}/images/val

# nc: 1  # number of classes
# names: ['tool']  # class names
# """

config_path = os.path.join(dataset_path, "data.yaml")
with open(config_path, "w") as file:
    file.write(config_content)

# Load a pre-trained YOLOv8 model
model = YOLO("yolov8s-seg.pt")

# Train the model with only 10 images instead of the full dataset
model.train(data=config_path, epochs=1, imgsz=640)

# Evaluate the model on the validation set
metrics = model.val()

Ultralytics YOLOv8.2.32 🚀 Python-3.12.3 torch-2.2.2 CPU (Apple M3)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/Volumes/Exodus/Data/ART-Net/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /Volumes/Exodus/Data/ART-Net/labels/train.cache... 662 images, 0 backgrounds, 0 corrupt: 100%|██████████| 662/662 [00:00<?, ?it/s]
val: Scanning /Volumes/Exodus/Data/ART-Net/labels/val.cache... 154 images, 0 backgrounds, 0 corrupt: 100%|██████████| 154/154 [00:00<?, ?it/s]


Plotting labels to runs/segment/train8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/segment/train8
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.015      2.255      3.378       1.31         51        640:  21%|██▏       | 9/42 [03:53<14:14, 25.89s/it]


KeyboardInterrupt: 

In [ ]:
if not os.path.exists("chkpts"):
    os.makedirs("chkpts")
# Save the model checkpoint to a file
chkpt = "chkpts/yolov8s-seg-artnet.pt"
model.save("chkpt")

In [ ]:
import matplotlib.pyplot as plt

losses = []
accuracies = []

# model.train(data=config_path, epochs=50, batch_size=8, imgsz=640, resume=checkpoint)
# Train the model for 50 more epochs but save new checkpoint to a file (and the metrics to a new file)
model = YOLO("chkpts/yolov8s-semiseg-artnet.pt")
for i in range(50):
    model.train(data=config_path, epochs=1, imgsz=640)
    losses.append(model.results["train"]["total_loss"])
    accuracies.append(model.results["val"]["mAP"])
    # Save the model checkpoint to a file
    checkpoint = model.save(f"yolov8s-semiseg-artnet-{i+1}.pt")

# Plot the training loss and validation mAP
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.subplot(1, 2, 2)
plt.plot(accuracies)
plt.title("Validation mAP")
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.tight_layout()
plt.show()

In [14]:
# results = model.track("/Volumes/Exodus/Data/6DOF 2023/Test 1/Dataset.mp4", tracker="bytetrack.yaml", save=True, show=True)

results = model.track(
    "data/6DOF/Dataset.mp4",
    tracker="bytetrack.yaml",
    save=True,
    show=True,
)

# results = model.track(
#     "/Volumes/Exodus/Data/EndoVis 2015/Tracking (Raw Video)/Dataset1/Video.avi",
#     tracker="bytetrack.yaml",
#     save=True,
#     show=True,
# )


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



KeyboardInterrupt: 

: 

In [12]:
import numpy as np
import torch

# Function to remove overlapping boxes using Non-Max Suppression
def non_max_suppression(boxes, scores, iou_threshold):
    if len(boxes) == 0:
        return []

    # Convert to numpy arrays
    boxes = np.array(boxes)
    scores = np.array(scores)

    # Compute areas of each box
    areas = (boxes[:, 2] - boxes[:, 0] + 1) * (boxes[:, 3] - boxes[:, 1] + 1)
    order = scores.argsort()[::-1]  # Sort by score in descending order

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        if order.size == 1:
            break
        xx1 = np.maximum(boxes[i, 0], boxes[order[1:], 0])
        yy1 = np.maximum(boxes[i, 1], boxes[order[1:], 1])
        xx2 = np.minimum(boxes[i, 2], boxes[order[1:], 2])
        yy2 = np.minimum(boxes[i, 3], boxes[order[1:], 3])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        inter = w * h

        iou = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(iou <= iou_threshold)[0]
        order = order[inds + 1]

    return keep


def process_image(image_path, file, model, iou_threshold=0.3, confidence_threshold=0.5, max_boxes=2):
    # Perform inference on a new image
    results = model(image_path, save=False, show=False, verbose=False)

    # Extract the results
    boxes = results[0].boxes.xyxy.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()

    # Step 1: Filter out boxes with confidence scores less than the threshold
    filtered_indices = np.where(scores >= confidence_threshold)[0]
    filtered_boxes = boxes[filtered_indices]
    filtered_scores = scores[filtered_indices]

    # Step 2: Remove overlapping boxes using Non-Max Suppression
    nms_indices = non_max_suppression(filtered_boxes, filtered_scores, iou_threshold)

    # Keep only the top boxes
    top_indices = nms_indices[:max_boxes]

    # Update results with filtered boxes
    results[0].boxes = results[0].boxes[top_indices]

    # Display the results
    # results[0].show()

    # Store image in tmp
    tmp = "tmp/" + file.replace(".bmp", "_tmp.png")
    results[0].save(tmp)

# test_dir = "/Volumes/Exodus/Data/6DOF 2023/Test 1/"
test_dir = "/Volumes/Exodus/Data/EndoVis 2015/Testing (Raw Images)/OP1/"

# For all bmp files
for file in sorted(os.listdir(test_dir)):
    if file.endswith(".png") and not file.startswith("._"):
        test_path = os.path.join(test_dir, file)
        process_image(test_path, file, model)

KeyboardInterrupt: 

In [ ]:
import imageio.v2 as imageio


# Combine all bmp files to video
# Create video from BMP files
def stitch_images(bmp_directory, output_video_path):
    # List all BMP files in the directory
    bmp_files = sorted(
        [
            os.path.join(bmp_directory, f)
            for f in os.listdir(bmp_directory)
            if f.endswith(".png") and not f.startswith(".")
        ]
    )

    # Create a video writer object
    with imageio.get_writer(output_video_path, fps=24) as writer:
        for file_path in bmp_files:
            image = imageio.imread(file_path)
            writer.append_data(image)

    print(f"Video saved at {output_video_path}")


output_image_name = "results/Test 1/yolo8-1.mp4"

# Stitch the images together
# stitch_images("tmp", output_image_name)

In [ ]:
# Use label file to draw original bounding box on original image
sample_label = "/Volumes/Exodus/Data/ART-Net/labels/val/Test_Pos_sample_0001.txt"
sample_image = (
    "/Volumes/Exodus/Data/ART-Net/Test/Test_Positive/Test_Pos_sample_0001.png"
)

# Draw plot with two images, one of sample image and one with results[0]
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
image = cv2.imread(sample_image)
with open(sample_label, "r") as lf:
    for line in lf:
        label = line.strip().split(" ")
    x_center, y_center, width, height = map(float, label[1:])
    image = cv2.imread(sample_image)
    x = int((x_center - width / 2) * image.shape[1])
    y = int((y_center - height / 2) * image.shape[0])
    w = int(width * image.shape[1])
    h = int(height * image.shape[0])
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")

# plt.subplot(1, 2, 2)
# save image
# results[0].save("tmp/results.png")
# # display image
# plt.imshow(cv2.cvtColor(cv2.imread("tmp/results.png"), cv2.COLOR_BGR2RGB))
# plt.axis("off")
# plt.show()